In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

with open("/kaggle/input/pride-prejudice-subtitles-and-text/PP.txt", "r", encoding="utf8") as f:
    data = f.read()

text = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ').replace('“', '').replace('”', '')
text = ' '.join(text.split())

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

with open('token.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle)

vocab_size = len(tokenizer.word_index) + 1
print(f"Taille du vocabulaire : {vocab_size}")

text = text[:100000]


Taille du vocabulaire : 6745


In [2]:
sequence_data = tokenizer.texts_to_sequences([text])[0]
sequences = []
for i in range(5, len(sequence_data)):
    words = sequence_data[i-5:i+1]
    sequences.append(words)

sequences = np.array(sequences)




In [3]:
# Diviser les données en entrée et sortie
X = sequences[:, :-1]  
y = sequences[:, -1]  
y = to_categorical(y, num_classes=vocab_size)





In [4]:
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Embedding(vocab_size, 40))
model.add(Dropout(0.3))
model.add(Bidirectional(GRU(120)))
model.add(Dropout(0.3))
model.add(Dense(vocab_size, activation='softmax'))
 

In [5]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=50, verbose=1)



Epoch 1/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.0323 - loss: 7.0711
Epoch 2/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0455 - loss: 6.0320
Epoch 3/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0616 - loss: 5.7833
Epoch 4/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0800 - loss: 5.5860
Epoch 5/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0958 - loss: 5.3633
Epoch 6/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1132 - loss: 5.1487
Epoch 7/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1199 - loss: 4.9448
Epoch 8/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1309 - loss: 4.7598
Epoch 9/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1433 - loss: 4.5449
Epoch 10/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1564 - loss: 4.3498
Epoch 11/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1671 - loss: 4.1734
Epoch 12/50
568/568 ━━━━━━━━━━━━━━━━━━━━

In [6]:
input_text = "She went to the store"
predict_next_words = 45


for _ in range(predict_next_words):
    input_sequence = tokenizer.texts_to_sequences([input_text])[0]
    
    input_sequence = input_sequence[-5:]
    
    input_sequence = np.array(input_sequence).reshape(1, -1)
    
    predicted_prob = model.predict(input_sequence, verbose=0)
    predicted_index = np.argmax(predicted_prob, axis=-1)
    
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    
    if output_word == "":
        print("Aucun mot prédit trouvé.")
        break
    
    input_text += " " + output_word

print(input_text)

She went to the store disposition you have material ill off good a most forbidding disagreeable man and being unworthy to be compared with his friend mr bingley had soon made himself acquainted with all the principal people in the room he replied he has no wish to know that
